In [89]:
!pip3 install transformers
!pip3 install torch
!pip3 install huggingface_hub
!pip3 install flash_attn
!pip3 install accelerate
!pip3 install datasets
!pip3 install evaluate
!pip3 install numpy

In [3]:
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
cache_name = ".depsek"

In [90]:
from transformers import AutoModel, AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from huggingface_hub import snapshot_download, notebook_login
import torch
from datasets import load_dataset
import numpy as np
import evaluate

In [5]:
notebook_login()

In [54]:
snapshot_dir = snapshot_download(repo_id=model_name, cache_dir=cache_name)
print(snapshot_dir)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

.depsek/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Instruct/snapshots/e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11


In [83]:
model_base = AutoModel.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [55]:
import sys
sys.path.append(".depsek/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Instruct/snapshots/")
!touch {snapshot_dir}/__init__.py
!ls {snapshot_dir}

config.json			   modeling_deepseek.py
configuration_deepseek.py	   model.safetensors.index.json
generation_config.json		   __pycache__
__init__.py			   README.md
model-00001-of-000004.safetensors  tokenization_deepseek_fast.py
model-00002-of-000004.safetensors  tokenizer_config.json
model-00003-of-000004.safetensors  tokenizer.json
model-00004-of-000004.safetensors


In [57]:
from e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11 import modeling_deepseek

In [63]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)
config.hidden_size = 100

In [84]:
model_classify = modeling_deepseek.DeepseekV2ForSequenceClassification(config)
model_classify.model = model_base

In [73]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [79]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [78]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [91]:
metric = evaluate.load("accuracy")

In [92]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [93]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [94]:
trainer = Trainer(
    model=model_classify,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

NameError: name 'small_train_dataset' is not defined

In [ ]:
trainer.train()